In [ ]:
import os
from datetime import datetime
import glob

import numpy as np

from lmsiage.uncertainty import ComputeObsUncertainty

from lmsiage.zarr_index_tools import cleanup_missing_files
cleanup_missing_files()

In [ ]:
mesh_dir = 'NERSC_arctic25km_sea_ice_age_v2p1/zarr/mesh'
unc_dir = 'NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert'

sid_dir = 'OSISAF_ice_drift_CDR_postproc'
obs_files = sorted(glob.glob(f'{sid_dir}/*/ice_drift*npz'))
obs_dates = [datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for ifile in obs_files]
print(f'Found {len(obs_files)} ice drift files from {obs_dates[0]} to {obs_dates[-1]}')

mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']

In [ ]:
csidu = ComputeObsUncertainty(obs_files, obs_dates, mesh_dir, unc_dir, xc, yc)
for year in range(1991, 2025):
    csidu.compute_obs_uncertainty(year)

In [ ]:
import matplotlib.pyplot as plt
from lmsiage.mesh_file import MeshFile

date = datetime(2024, 12, 31)

unc_file = date.strftime('NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/%Y/unc_%Y%m%d.zip')
mesh_file = date.strftime('NERSC_arctic25km_sea_ice_age_v2p1/zarr/mesh/%Y/mesh_%Y%m%d.zip')
print(MeshFile(unc_file).read_names())

unc_sid, unc_sic = MeshFile(unc_file).load(['unc_sid', 'unc_sic'], as_dict=False)
x, y, t = MeshFile(mesh_file).load(['x', 'y', 't'], as_dict=False)
plt.tripcolor(x, y, t, unc_sic)
plt.colorbar()
plt.show()
plt.tripcolor(x, y, t, unc_sid)
plt.colorbar()
plt.show()